# Exercice : Perceptron simple

Un perceptron est un modèle très simple (voir le plus simple) de reseaux de neurones. Il consiste simplement en une couche d'entrée (les inputs) et directement une couche de sortie (output) sans couches cachées entre les deux.

![Illustration d'un peceptron](https://miro.medium.com/v2/resize:fit:256/format:webp/1*JLlHNhYjyY9h9y1D1sy8Zw.png)

Nous allons entrainer notre modèle afin de classifier les images du dataset MNIST:

![Illustration du dataset MNIST](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Ce dataset est composé d'images de 28x28 representant des chiffres, le but va alors etre de classifier les images dans la bonne categorie.


##Imports

Afin de faciliter notre implementation, nous allons principalement utilisé la librairie Numpy, qui permet de gerer facilement des arrays en n dimensions.

Pour de l'aide avec les operations sur les ndarray, appelez nous, [suivez ce tutoriel](https://numpy.org/doc/stable/user/quickstart.html), ou [cherchez dans la documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)/google.

In [ ]:
!pip install einops > /dev/null

import math
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from einops import rearrange

## Fonctions pour load les données

Ces fonctions mettent au bon format les données en entré

In [ ]:
(train_src, train_tgt), (test_src, test_tgt) = mnist.load_data()
train_src = rearrange(train_src,'i w h -> i (w h)')/255
test_src = rearrange(test_src,'i w h -> i (w h)')/255

In [ ]:
train_labels_one_hot = np.zeros((len(train_tgt), 10))
train_labels_one_hot[range(len(train_tgt)), train_tgt] = 1
train_tgt = train_labels_one_hot

In [ ]:
test_labels_one_hot = np.zeros((len(test_tgt), 10))
test_labels_one_hot[range(len(test_tgt)), test_tgt] = 1
test_tgt = test_labels_one_hot

In [ ]:
def load_by_batches(dataset, batch_size):
    for i in range(math.ceil(dataset.shape[0] / batch_size)):
        yield dataset[i * batch_size:i * batch_size + batch_size]

### Model et code à completer

Voici le code a completer, ce code est separé en 3 étapes:


*   L'initialisation du model
*   La passe avant
*   La passe arrière

Pour les passes, toutes les formules sont dans les slides a différents endroits, à vous de retrouver les bonnes forumules et les mettre au bon endroit.

Vous avez peut etre remarqué l'utilisation d'une variable batch_size, si cela vous pose des problème, ou que vous voulez plus d'informations, demendez, c'est une partie qui a volontairment été laissée de coté afin de pratiquer plus rapidement.



In [ ]:
class Model:

    def __init__(self, input_size, output_size):
        # TODO #1 : Initialiser le model
        self.weight = None
        self.bias = None

        self.learning_rate = None
        self.batch_size = 16

    def forward(self, x):
        # TODO #2 : implementer la passe vers l'avant
        pass

    def backpropagate(self,x,output,tgt):
        # TODO #3 : implementer la passe vers l'arriere
        pass

Ici, vous retrouvez les quelques fonction mentionnées dans les slides, certaines peuvent vous etres utiles dans l'exercice

In [ ]:
# Fonction utiles

def softmax( x):
    x = np.exp(x)
    return x / x.sum(axis=1, keepdims=True)

def compute_accuracy(output, tgt):
    acc = tgt.argmax(axis=1) - output.argmax(axis=1)
    acc = acc.shape[0] - np.count_nonzero(acc)
    return acc

def compute_loss(output, tgt):
    return -np.log(np.max(output * tgt, axis=1)).sum()

### Training du model

In [ ]:
def train_model(model, src, tgt, src_test, tgt_test):
    epoch = 1

    acc = []
    loss = []

    acc_test = []
    loss_test = []
    while True:
        print(f"Epoch #{epoch}")

        acc.append(0)
        loss.append(0)

        for x, y in zip(load_by_batches(src, model.batch_size), load_by_batches(tgt, model.batch_size)):

            output = model.forward(x)
            assert output.shape == y.shape
            acc[-1] += compute_accuracy(output, y)
            l = compute_loss(output, y)
            loss[-1] += l

            model.backpropagate(x, output, y)
        acc[-1] /= src.shape[0]
        loss[-1] /= src.shape[0]

        output = model.forward(src_test)
        acc_test.append(compute_accuracy(output, tgt_test))
        l = compute_loss(output, tgt_test)
        loss_test.append(l)
        acc_test[-1] /= src_test.shape[0]
        loss_test[-1] /= src_test.shape[0]

        print(f'Train| Accuracy: {acc[-1]:.3} - Loss: {loss[-1]:.3}')
        print(f'Test| Accuracy: {acc_test[-1]:.3} - Loss: {loss_test[-1]:.3}')
        print()

        if epoch % 10 == 0:
            fig, axs = plt.subplots(2, 2)

            axs[0, 0].plot(acc)
            axs[1, 0].plot(loss)
            axs[0, 1].plot(acc_test, 'tab:orange')
            axs[1, 1].plot(loss_test, 'tab:orange')

            axs[0, 0].grid(True)
            axs[1, 0].grid(True)
            axs[0, 1].grid(True)
            axs[1, 1].grid(True)

            axs[0, 0].set_title('Train dataset')
            axs[0, 1].set_title('Test dataset')
            axs[0, 0].set(ylabel='Accuracy')
            axs[1, 0].set(ylabel='Loss')

            plt.show()
        epoch += 1

In [ ]:
model = Model(train_src.shape[1], train_tgt.shape[1])
train_model(model, train_src, train_tgt, test_src, test_tgt)